# Requesting raw data from WebCTRL

This notebooks provides the minimum necessary code to access data from WebCTRL's API, the software currently used in the Frogs Project. For an overview of the architecture, see the readme.md file of the project where is notebook is hosted. 

## 1. Required Input Parameters

### 1.1 Host

Currently WebCTRL API (BulkTrendServer add-on), is hosted in HNEI.

In [8]:
host = 'http://www.soest.hawaii.edu/hneienergy/bulktrendserver/read'

### 1.2 Authentication

Data is only available through authentication. Ask a member of ERDL for username and password.

In [13]:
with open('auth.txt', 'r') as f:
    api_user = f.readline().rstrip()
    api_pass = f.readline().rstrip()

### 1.3 Sensor

A list of sensor IDs is currently only available through WebCTRL API interface. In the future, the list of IDs will be added to this repository. Ask a member of ERDL for the IDs of interest. 

The sensor id format is as shown in the Lookup ID column of the trend search page. An example of this format is **ABSPATH:1:#frog1_duct_sensors_1/m030**.



In [16]:
sensor_ids = ['ABSPATH:1:#frog1_duct_sensors_1/m030','ABSPATH:1:#frog1_room_sensors_1/zone_humidity_1_tn']

### 1.4 Start and End Date

 * **Start Date**: yyyy-mm-dd, for example: "2017-07-16". The starting date is inclusive.
 
 * **End Date**: yyyy-mm-dd, i.e., same as start date. Data points will be retrieved through the end of the specified end date (exclusive of the midnight transitioning to the next day). Midnight is considered to be part of the following day. So to retrieve only one day's trends, specify the same date for start and end.

In [11]:
start_date = '2017-09-13'
end_date = '2017-09-13'

### 1.5 Output Format (optional)

The output format of the data can either be:

 * csv
 * json (default)

In [12]:
output_format = 'json'

## 2. HTTP Request

In [24]:
import requests

params = {'id': sensor_ids, 'start': start_date, 'end': end_date, 'format': output_format}
auth = auth = (api_user, api_pass)

request = requests.post(host, params = params, auth=tuple(auth))

## 3. Output Format

In the JSON format, the overall result is an array of sensor id objects. Each sensor id object has:

 * An id with key of "id" 
 * An array of samples with a key of "s". 
    * Each sample in the array has a timestamp with a key of "t" and a value. The key for the value depends on the type of data. Analog values have a key of "a" and digital values have a a key of "d". The string encoding for both dates and values is identical to what is used in the CSV format.
    
**Note that the order keys are written into an object is not guaranteed.**

Example encoding for two sources with two samples each. Note that this has been broken into multiple lines and white space has been added for clarity. The real result has no whitespace and is all on one line:

```
[
  {
    "id":"ABSPATH:1:#board_room/ht_stpt_tn",
    "s":[{"t":"1282017600000","a":"55.1"},{"t":"1282017900000","a":"55"}]
  },
  {
    "id":"ABSPATH:1:#board_room/locked_tn",
    "s":[{"t":"1282017600000","d":"false"},{"t":"1282017900000","d":"true"}]
  }
]
```

### 3.1 Request's Status

In [33]:
def getStatusCode(request):
    return( (request.status_code,requests.status_codes._codes[request.status_code][0]) )

print(getStatusCode(request))

(200, 'ok')


### 3.2 Request's Readings

In [63]:
sensors = req.json()

def getSensorIDIndexes(request):
    sensor_indexes = {sensors[i]["id"]: i for i in range(len(sensors))}
    return(sensor_indexes)

sensor_indexes = getSensorIDIndexes(request)

{'ABSPATH:1:#frog1_duct_sensors_1/m030': 0, 'ABSPATH:1:#frog1_room_sensors_1/zone_humidity_1_tn': 1}


In [65]:
def getReadingsFromSensor(sensor_id,sensor_indexes):
    sensor_index = sensor_indexes[sensor_id]
    return(sensors[sensor_index]["s"])

import pprint
pprint.pprint(getReadingsFromSensor('ABSPATH:1:#frog1_duct_sensors_1/m030',sensor_indexes))

[{'a': '55.2', 't': 1505296800000},
 {'a': '55.3', 't': 1505297100000},
 {'a': '55.5', 't': 1505297400000},
 {'a': '55.6', 't': 1505297700000},
 {'a': '55.6', 't': 1505298000000},
 {'a': '55.9', 't': 1505298300000},
 {'a': '56.3', 't': 1505298600000},
 {'a': '56.6', 't': 1505298900000},
 {'a': '56.6', 't': 1505299200000},
 {'a': '56.7', 't': 1505299500000},
 {'a': '56.8', 't': 1505299800000},
 {'a': '56.8', 't': 1505300100000},
 {'a': '56.8', 't': 1505300400000},
 {'a': '56.9', 't': 1505300700000},
 {'a': '57', 't': 1505301000000},
 {'a': '57', 't': 1505301300000},
 {'a': '57', 't': 1505301600000},
 {'a': '57.1', 't': 1505301900000},
 {'a': '57.3', 't': 1505302200000},
 {'a': '57.3', 't': 1505302500000},
 {'a': '57.3', 't': 1505302800000},
 {'a': '57.4', 't': 1505303100000},
 {'a': '57.3', 't': 1505303400000},
 {'a': '57.2', 't': 1505303700000},
 {'a': '57.5', 't': 1505304000000},
 {'a': '57.6', 't': 1505304300000},
 {'a': '57.6', 't': 1505304600000},
 {'a': '57.7', 't': 1505304900000}

## 4. References

The official documentation for the WebCTRL API, BulkTrendServer, is available [on the associated Github project](https://github.com/alclabs/BulkTrendServer), in particular [the doc file](https://github.com/alclabs/BulkTrendServer/blob/master/src/main/webapp/doc.html). 

A copy of the same doc file is available in the repository this notebook is hosted (i.e. ERDL / sensors project) in case the original source becomes unavailable. 